In [128]:
import os
import pandas as pd
import seaborn as sns

import sys  # noqa
sys.path.append('../..')  # noqa

from utils.data_paths import make_data_list
from dataclasses import dataclass, field, asdict
from datetime import datetime, timedelta
from typing import Tuple
import re


In [129]:
@dataclass
class OutputLogs:
    """ Holds the paths to the output logs for each pipeline. """
    bio3: str = ""
    bio4: str = ""
    jams: list[str] = field(default_factory=list)
    wgsa2: str = ""
    woltka: list[str] = field(default_factory=list)
    sunbeam: str = ""


@dataclass
class TimeData:
    """ Holds the average or total time for each pipeline."""
    bio3: timedelta = timedelta()
    bio4: timedelta = timedelta()
    jams: timedelta = timedelta()
    wgsa2: timedelta = timedelta()
    woltka: timedelta = timedelta()
    sunbeam: timedelta = timedelta()


@dataclass
class ThreadData:
    """ Holds the number of CPUS used in each run. """
    bio3: int = 0
    bio4: int = 0
    jams: int = 0
    wgsa2: int = 0
    woltka: int = 0
    sunbeam: int = 0


In [130]:
def search_for_files(path: str):
    """ Look for anadama.log, JAMS logs, WGSA logs, Wolka, and Sunbeam logs."""
    data_obj = OutputLogs()

    for root, dirs, files in os.walk(path):
        # print(files)
        for f in files:
            if f == "anadama.log":
                if "bio4" in root:
                    data_obj.bio4 = os.path.join(root, f)
                elif "bio3" in root:
                    data_obj.bio3 = os.path.join(root, f)
            elif f.endswith("JAMS.log"):
                # Don't add the beta log or the negative control log.
                if "beta" in root or "Neg" in f:
                    continue
                else:
                    data_obj.jams.append(os.path.join(root, f))
            elif f == "logfile.txt":
                data_obj.wgsa2 = os.path.join(root, f)

            elif f == "classify_time.log" or f == "bowtie_time.log":
                data_obj.woltka.append(os.path.join(root, f))

            # elif re.match("\d+_benchmarks.tsv", f):
            #     data_obj.sunbeam = os.path.join(root, f)

            elif f == "sunbeam.log":
                data_obj.sunbeam = os.path.join(root, f)

    return data_obj


In [131]:
anadama_format = "%Y-%m-%d %H:%M:%S,%f"
wanted_time_fmt = "%H:%M:%S"


def parse_bio_time(log_path: str):
    """Parse the anadama.log file to get the start and end times."""
    # First line is the start time, last line is the end time.

    with open(log_path, 'r') as f:
        lines = f.readlines()
        start = datetime.strptime(
            lines[0].strip().split('\t')[0], anadama_format)
        end = datetime.strptime(
            lines[-1].strip().split('\t')[0], anadama_format)

        # Find the line that has "threads" in it.
        threads = 0
        for line in lines:
            if "threads" in line:
                # Get the number of threads used.
                threads = int(line.split(' ')[-1])
                break

        elapsed = end - start

        return elapsed, threads


In [132]:
jams_format_time = "%Y-%m-%d %H:%M:%S"


def average_time(times: list):
    """ Get the average time for each pipeline. """
    average = sum(times, timedelta()) / len(times)
    return average


def parse_jams_time(logs: list):
    """ Parse all of the JAMS logs. """

    cpu_regex = "Saving project workspace image using fastSave package with \d+ CPUs"

    times = []

    # They all ran with the same number of threads, so we can overwrite this in the for loop.
    threads = 0
    for l in logs:
        # open the file
        with open(l, 'r') as f:
            lines = f.readlines()
            # First line is start time.
            start = " ".join(lines[0].strip().split()[1:3]).strip("[]")
            start_time = datetime.strptime(start, jams_format_time)
            # Last line is end time.
            end = " ".join(lines[-1].strip().split()[1:3]).strip("[]")
            end_time = datetime.strptime(end, jams_format_time)

            for line in lines:
                if "Saving project workspace image using fastSave package with" in line:
                    threads = int(line.split()[-2])
                    break

            elapsed = end_time - start_time

            times.append(elapsed)

    return average_time(times), threads


In [133]:
def parse_wgsa2(file_path: str):
    """ Parse the WGSA output log for the time."""
    with open(file_path, 'r') as f:
        lines = f.readlines()
        start = " ".join(lines[0].strip().split()[0:2]).strip("[]")
        start_time = datetime.strptime(start, anadama_format)
        # Last line is end time.
        end = " ".join(lines[-1].strip().split()[0:2]).strip("[]")
        end_time = datetime.strptime(end, anadama_format)

        threads = 0
        for line in lines:
            if 'Provided cores:' in line:
                threads = int(line.split()[-1])
                break

        elapsed = end_time - start_time
        return elapsed, threads


In [134]:
def parse_woltka_time(logs: list):
    times = []
    for log in logs:
        # Second line gives column widths.
        with open(log, 'r') as f:
            dash_line = f.readlines()[1].strip().split()
            widths = [len(x)+1 for x in dash_line]

            df = pd.read_fwf(log, widths=widths, skiprows=[1], header=0)
            df = df.loc[df["JobName"] == "swarm"]

            # Split along the colon. This is of the format DD:HH:MM.
            df["Elapsed"] = df["Elapsed"].str.split(":").apply(
                lambda x: timedelta(days=int(x[0]), hours=int(x[1]), minutes=int(x[2])))

            avg_td = df["Elapsed"].mean()

            times.append(avg_td)

    # Sum of the bowtie and classify times.
    total_time = sum(times, timedelta())
    return total_time.to_pytimedelta()


In [135]:
def parse_sunbeam(file_path: str) -> Tuple[timedelta, int]:
    # We will read the sunbeam.log file. We want lines that match the following expression: [Tue Apr 11 13:59:32 2023].

    # We will use a regex to match the date and time.
    regex = "\[\w{3} \w{3} \d{2} \d{2}:\d{2}:\d{2} \d{4}\]"
    with open(file_path, 'r') as f:
        lines = f.readlines()
        times = [re.sub("[\[\]]", "", x.strip())
                 for x in lines if re.match(regex, x)]
        start, end = times[0], times[-1]

        start_time, end_time = datetime.strptime(
            start, "%a %b %d %H:%M:%S %Y"), datetime.strptime(end, "%a %b %d %H:%M:%S %Y")

        return end_time - start_time, 32

# parse_sunbeam("/Volumes/TBHD_share/valencia/pipelines/microbio_spectrum/CLEANED/pipelines/sunbeam4/sunbeam.log")


In [136]:
def parse_all_times(log_paths: OutputLogs) -> TimeData:
    """ Parses all time data, returns a TimeData object."""
    time_per_cpu = TimeData()
    raw_times = TimeData()
    # threads_data = ThreadData()

    # Parse bio3 time.
    # bio3_time, threads = parse_bio_time(log_paths.bio3)
    # times.bio3 = bio3_time / threads
    # threads_data.bio3 = threads
    # Parse bio4 time.
    bio4_time, threads = parse_bio_time(log_paths.bio4)
    raw_times.bio4 = bio4_time
    time_per_cpu.bio4 = bio4_time / threads
    # threads_data.bio4 = threads
    # Parse JAMS time.
    jams_times, threads = parse_jams_time(log_paths.jams)
    raw_times.jams = jams_times
    time_per_cpu.jams = jams_times / threads
    # threads_data.jams = threads
    # Parse WGSA2 time.
    # wgsa2_time, threads = parse_wgsa2(log_paths.wgsa2)
    # times.wgsa2 = wgsa2_time / threads
    # # threads_data.wgsa2 = threads
    # # Parse Woltka time.
    # woltka_times = parse_woltka_time(log_paths.woltka)
    # # Woltka used 16 threads. This could be automated but since we are doing just this one pipeline, it's fine.
    # times.woltka = woltka_times / 16

    sunbeam_times = parse_sunbeam(log_paths.sunbeam)
    raw_times.sunbeam = sunbeam_times[0]
    time_per_cpu.sunbeam = sunbeam_times[0] / sunbeam_times[1]

    return time_per_cpu, raw_times


def save_raw_time(raw_time: TimeData, community_name: str):
    """ Saves the raw time data to a file after converting to HH:MM:SS """

    df = pd.DataFrame(asdict(raw_time), index=[0])

    df = df.applymap(lambda x: str(x).split(".")[0])

    df.index = [community_name]

    df.to_csv(os.path.join("raw_time.csv"), mode='a')


def analyze_times(log_paths: OutputLogs, community_name: str):
    time_per_cpu, raw_time = parse_all_times(log_paths)

    save_raw_time(raw_time, community_name)

    time_df = pd.DataFrame(asdict(time_per_cpu), index=[0])
    # Drop the second row.
    time_df.drop(["bio3", "wgsa2", "woltka"], axis=1, inplace=True)

    # display(time_df)

    # Relative difference is (x2 - min) / min
    relative_times = time_df.apply(lambda x: (
        x - time_df.min(axis=1)) / time_df.min(axis=1) * 100)

    # Set index value to be: "Relative Time (Factor of Smallest Time)"
    relative_times.index = ["Relative Time Per CPU (%)"]

    return time_df, relative_times


In [137]:
paths_dict = {
    # "nist": "/Volumes/TBHD_share/valencia/pipelines/NIST/"
    "tourlousse": "/Volumes/TBHD_share/valencia/pipelines/microbio_spectrum/CLEANED/pipelines/",
    "hilo": "/Volumes/TBHD_share/valencia/pipelines/amos/nibsc/hilo/",
    "mixed": "/Volumes/TBHD_share/valencia/pipelines/amos/nibsc/mixed/",
    "mbarc": "/Volumes/TBHD_share/valencia/pipelines/MBARC/pipelines/"
}


def main():
    if os.path.exists("raw_time.csv"):
        os.remove("raw_time.csv")

    final_df = pd.DataFrame()

    for k in paths_dict.keys():
        log_paths = search_for_files(paths_dict[k])
        # print(log_paths)

        time_df, relative_time_df = analyze_times(log_paths, k)

        # display(time_df)

        # Format timedf to HH:MM:SS
        time_df = time_df.applymap(lambda x: str(x).split(".")[0])

        # Strip the days from the raw time_df.
        time_df = time_df.applymap(lambda x: x.split(" ")[-1])
        time_df.index = [k]

        # Concat the two dataframes.
        final_df = pd.concat([time_df, final_df], axis=0)

        display(time_df)

    final_df.to_latex("time_table.tex", index=True, escape=True)


main()


,bio4,jams,sunbeam
tourlousse,00:02:26,00:02:35,00:03:49


,bio4,jams,sunbeam
hilo,00:02:05,00:01:57,00:01:11


,bio4,jams,sunbeam
mixed,00:02:11,00:02:48,00:01:17


,bio4,jams,sunbeam
mbarc,00:23:10,00:44:14,01:17:57


/var/folders/22/_2115q192kx7c1z764lwhkth898844/T/ipykernel_16916/1740234926.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  final_df.to_latex("time_table.tex", index=True, escape=True)
